### From a File


In [ ]:
from main import create_taxonomy

filename = "cais_data_expanded.csv"
brand_terms = [
    "cais",
    "cais group",
    "glas",
    "glas funds",
    "halo",
    "halo investing",
    "icapital",
    "icapital network",
]
website_subject = "Alternate Investing Platform"

taxonomy, df, samples = create_taxonomy(
    filename,
    text_column="keyword",
    search_volume_column="search_volume",
    min_df=5,
    brand_terms=brand_terms,
)

df.to_csv("cais_data_taxonomy.csv", index=False)

print("\n".join(taxonomy))

In [ ]:
from main import create_taxonomy

filename = "HM Raw Data.csv"
website_subject = "Houston Methodist Hospital"

brand_terms = [
    "luke",
    "lukes",
    "md anderson",
    "anderson",
    "hca",
    "stlukes",
    "memorial hermann",
    "hermann",
    "herman",
    "houston methodist",
    "methodist",
    "st joseph",
    "joseph",
]

taxonomy, df, samples = create_taxonomy(
    filename,
    website_subject=website_subject,
    text_column="keyword",
    search_volume_column="search_volume",
    min_cluster_size=10,
    min_samples=3,
    limit_queries=3,
    ngram_range=(1, 5),
    min_df=10,
    brand_terms=brand_terms,
)


df.to_csv("HM_Raw_Data_ngram_taxonomy2.csv", index=False)
print("\n".join(taxonomy))

### From a GSC Account

In [1]:
from main import create_taxonomy

brand_terms = ["ledgeloungers", "ledge"]
property = "sc-domain:ledgeloungers.com"
website_subject = "Ledge Lounger offers luxury pool & outdoor furniture designed to create perfect spaces for outdoor entertaining and relaxation."

taxonomy, df, samples = create_taxonomy(
    property,
    website_subject=website_subject,
    days=30,
    ngram_range=(1, 5),
    cross_encoded=True,
    min_df=5,
    brand_terms=brand_terms,
    limit_queries_per_page=5,
)


df.to_csv("ledgeloungers_taxonomy2.csv", index=False)

df.head()

c:\Users\jroak\anaconda3\envs\taxonomyml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-21 08:59:41.047 | INFO     | lib.searchconsole:load_gsc_account_data:51 - Creating dataframe...
2023-07-21 08:59:59.075 | INFO     | lib.searchconsole:load_gsc_account_data:58 - Dataframe created.
2023-07-21 08:59:59.410 | INFO     | main:create_taxonomy:215 - Got Data. Dataframe shape: (2419, 7)
2023-07-21 08:59:59.412 | INFO     | main:create_taxonomy:217 - Filtering Query Data.
2023-07-21 08:59:59.500 | INFO     | main:score_and_filter_df:110 - Got ngram frequency. Dataframe shape: (384, 2)
2023-07-21 08:59:59.555 | INFO     | main:score_and_filter_df:113 - Merged Ngrams. Dataframe shape: (321, 4)
2023-07-21 09:00:01.524 | INFO     | main:score_and_filter_df:143 - Final score and filter length: 321
2023-07-21 09:00:

   ngram_size    query  merged_frequency  frequency  search_volume     score
0           1  lounger          1.000000   1.000000       1.000000  1.000000
1           1     pool          0.154122   0.690695       0.836386  0.495254
2           1     hole          0.014337   0.013106       0.560620  0.287479
3           1    table          0.111111   0.328965       0.423304  0.267208
4           1  outdoor          0.301075   0.250328       0.218655  0.259865


2023-07-21 09:00:10.897 | INFO     | main:create_taxonomy:234 - Reviewing OpenAI's work.
2023-07-21 09:00:17.030 | INFO     | main:create_taxonomy:254 - Getting structure.
2023-07-21 09:00:17.031 | INFO     | main:create_taxonomy:258 - Adding categories.
2023-07-21 09:00:17.033 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 09:00:17.965 | INFO     | lib.clustering:fit_pairwise_crossencoded:479 - Getting embeddings.
2023-07-21 09:00:17.967 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 09:00:17.969 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
Batches: 100%|██████████| 22/22 [00:02<00:00,  7.50it/s]
2023-07-21 09:00:21.442 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 09:00:21.443 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
2023-07-21 09:00:22.073 | INFO     | lib.clustering:fit_

PermissionError: [Errno 13] Permission denied: 'ledgeloungers_taxonomy.csv'

In [2]:
df.to_csv("ledgeloungers_taxonomy2.csv", index=False)

In [9]:
import pandas as pd
from lib.clustering import ClusterTopics

df = pd.read_csv("ledgeloungers_taxonomy.csv")

df.head()

queries = list(set(df["query"].tolist()))
queries = [queries[idx] for idx in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]]
taxonomies = list(set(df["taxonomy"].tolist()))

cluster_model = ClusterTopics(cluster_categories=taxonomies)
cluster_model.corpus = queries
cluster_model.embeddings = cluster_model.get_embeddings(queries)


2023-07-21 07:28:42.942 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 07:28:42.944 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2


2023-07-21 07:28:43.519 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 07:28:43.521 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

category_embeddings = cluster_model.get_embeddings(taxonomies)
query_embeddings = cluster_model.embeddings

query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
category_embeddings = category_embeddings / np.linalg.norm(category_embeddings, axis=1, keepdims=True)


cosine_similarity_matrix = cosine_similarity(query_embeddings, category_embeddings)


2023-07-21 07:28:49.580 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 07:28:49.582 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2


In [ ]:
idxs = 10
top_n = 5
import settings
from tqdm import tqdm
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/qnli-electra-base")
cross_encoder_pairs = []


for idx, query in enumerate(queries):
    #print("#" * 50)
    #print(f"Query: {query}")

    #print("\nCosine Similarity:")
    top_categories = np.argsort(cosine_similarity_matrix[idx])[-top_n:][::-1]
    pairs = []
    for i in top_categories:
        #print(taxonomies[i], cosine_similarity_matrix[idx][i])
        pairs.append((query, taxonomies[i]))
    
    cross_encoder_pairs.append(pairs)
    #print()


cross_encoder_similarity = np.array([
            cross_encoder.predict(pairs).flatten()
            for pairs in cross_encoder_pairs
        ])

for idx, query in enumerate(queries):
    print("#" * 50)
    print(f"Query: {query}")

    print("\nCross-encoder Similarity:")
    
    # Test cross_encoder_similarity scores for this query.  If all scores are high and similar, then the query is ambiguous.  If on score is considerably lower than the others, then the query is not ambiguous.
    


    top_category= np.argmin(cross_encoder_similarity[idx])
    print(cross_encoder_pairs[idx][top_category][1], cross_encoder_similarity[idx][top_category])
    print()



In [15]:
import settings
from tqdm import tqdm
from sentence_transformers import CrossEncoder
import numpy as np

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

query = "in the pool chairs"

options = """Chairs Pool Chairs
Pool Furniture Chairs
Pool Furniture Loungers
Loungers Pool Loungers
Pool Furniture Accessories""".split("\n")

options = [o.lower() for o in options]

cross_encoder_pairs = [(query, option) for option in options]

cross_encoder_similarity = cross_encoder.predict(cross_encoder_pairs).flatten()

print("Query:", query)
print("Options:")
for idx, pair in enumerate(cross_encoder_pairs):
    print(pair[1], cross_encoder_similarity[idx])

print()
print(f"Best: {cross_encoder_pairs[np.argmax(cross_encoder_similarity)][1]}, {cross_encoder_similarity[np.argmax(cross_encoder_similarity)]}")


Query: in the pool chairs
Options:
chairs pool chairs 7.287129
pool furniture chairs 7.0691895
pool furniture loungers 3.595707
loungers pool loungers -2.3364563
pool furniture accessories 2.3251443

Best: chairs pool chairs, 7.287128925323486


In [1]:
from main import add_categories
import pandas as pd 

df = pd.read_csv("ledgeloungers_taxonomy2.csv")

structure = list(set(df.taxonomy.tolist()))

# Random sample of 10 df rows
df = df.sample(20)

df = add_categories(structure, df, 
                    cross_encoded=True, 
                    percentile_threshold=50,
                    std_dev_threshold= 0.1)




c:\Users\jroak\anaconda3\envs\taxonomyml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-21 10:12:44.159 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 10:12:44.160 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
2023-07-21 10:12:45.330 | INFO     | lib.clustering:fit_pairwise_crossencoded:479 - Getting embeddings.
2023-07-21 10:12:45.331 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 10:12:45.333 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: all-MiniLM-L6-v2
2023-07-21 10:12:45.903 | INFO     | lib.clustering:get_embeddings:100 - Using local embeddings
2023-07-21 10:12:45.904 | INFO     | lib.clustering:get_embeddings:107 - Using custom embedding model: a

In [2]:
df[['query', 'taxonomy']]

,query,taxonomy
1390,tanning chairs,Miscellaneous
1334,luxury pools,Miscellaneous
133,lounger cleaning,Pool Furniture > Loungers > Chaise Loungers
595,pool loungers,Pool Furniture > Loungers > Pool Loungers
528,outdoor umbrella with lights,Pool Furniture > Umbrellas > Cantilever Umbrellas
2296,pool lounger,Pool Furniture > Loungers > Pool Loungers
621,weather resistant cushions,Pool Furniture > Accessories > Cushions
439,lounger chaise,Pool Furniture > Loungers > Chaise Loungers
1006,flat cushion,Miscellaneous
2192,lounger signature chaise deep,Pool Furniture > Loungers > Signature Chaise L...


In [ ]:
from lib.api import get_openai_response_chat
import settings
from tqdm import tqdm

for url, row in tqdm(by_url.iterrows(), total=by_url.shape[0]):
    samples = "\n".join(row["keyword"][:10])

    prompt = f"""As an expert at understanding search intent, 
    Please provide a decrtiptive subject for the page based on the provided Search Queries.

    Search Queries:
    {samples}

    Subject: """

    explanation = get_openai_response_chat(
        prompt,
        model=settings.CLUSTER_DESCRIPTION_MODEL,
        system_message="You are an expert at understanding the intent of Google searches.",
    )

    samples = ", ".join(row["keyword"][:10])
    print(explanation)
    print(samples)
    print()